Load libraries

In [1]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from matplotlib import pyplot
import tensorflow as tf
import os
from time import time

Using TensorFlow backend.


Load Google Drive API

In [0]:
from google.colab import drive

Mouting Google Drive for storage access

In [3]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Path to Images location in Google Drive

In [0]:
CODE_PATH_DIR = "drive/My Drive/Disertatie/"
TRAIN_IMAGES_PATH = os.path.join(CODE_PATH_DIR, os.path.join("Train_Images", "Aerial/"))
OUTPUT_IMAGES_PATH = os.path.join(CODE_PATH_DIR, os.path.join("Output_Images", "Aerial/"))
LOGDIR = os.path.join(CODE_PATH_DIR, "LogDir")

Definig size for input

In [0]:
size = (256, 256)

Input filename

In [6]:
FILENAME = "map_test_" + str(size[0]) + ".npz"
FILENAME_OPEN_PATH = os.path.join(CODE_PATH_DIR, FILENAME)
print("Input filename: " + str(FILENAME))
print("Open path: " + str(FILENAME_OPEN_PATH))
print("LogDir: " + str(LOGDIR))

Input filename: map_test_256.npz
Open path: drive/My Drive/Disertatie/map_test_256.npz
LogDir: drive/My Drive/Disertatie/LogDir


Define the discriminator model

In [0]:
def define_discriminator(image_shape):
    print("Discriminator image_shape: " + str(image_shape))
    # Weight initialization
    init = RandomNormal(stddev=0.02)
    # Source image input
    in_src_image = Input(shape=image_shape)
    # Target image input
    in_target_image = Input(shape=image_shape)
    # Concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(1024, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

Define an encoder block

In [0]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # Weight initialization
    init = RandomNormal(stddev=0.02)
    # Add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # Conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # Leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

Define a decoder block

In [0]:
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # Weight initialization
    init = RandomNormal(stddev=0.02)
    # Add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # Add batch normalization
    g = BatchNormalization()(g, training=True)
    # Conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # Merge with skip connection
    
    g = Concatenate()([g, skip_in])
    # Relu activation
    g = Activation('relu')(g)
    
    return g

Define the standalone generator model

In [0]:
def define_generator(image_shape = (256,256,3)):
    print("Generator image_shape: " + str(image_shape))
    # Weight initialization
    init = RandomNormal(stddev=0.02)
    # Image input
    in_image = Input(shape=image_shape)
    
    # Encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    
    # Bottleneck, no batch norm and relu
    b = Conv2D(1024, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    
    # Decoder model
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512, dropout=False)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # Output
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # Define model
    model = Model(in_image, out_image)
    return model

Define the combined generator and discriminator model, for updating the generator

In [0]:
def define_gan(g_model, d_model, image_shape):
	  # Make weights in the discriminator not trainable
    d_model.trainable = False
    # Define the source image
    in_src = Input(shape=image_shape)
    # Connect the source image to the generator input
    gen_out = g_model(in_src)
    # Connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # Src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # Compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

Load and prepare training images

In [0]:
def load_real_samples(filename):
	# Load compressed arrays
	data = load(filename)
	# Unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# Scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X2, X1]

Select a batch of random samples, returns images and target

In [0]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# Unpack dataset
	trainA, trainB = dataset
	# Choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# Retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# Generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

Generate a batch of images, returns images and targets

In [0]:
def generate_fake_samples(g_model, samples, patch_shape):
	# Generate fake instance
	X = g_model.predict(samples)
	# Create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

Generate samples and save as a plot and save the model

In [0]:
def summarize_performance(step, g_model, dataset, n_samples=3):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	# plot real source images
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realA[i])
	# plot generated target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_fakeB[i])
	# plot real target image
	for i in range(n_samples):
		pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
		pyplot.axis('off')
		pyplot.imshow(X_realB[i])
	# save plot to file
	filename1 = 'plot_%06d.png' % (step+1)
	pyplot.savefig(CODE_PATH_DIR + filename1, dpi = 600)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%06d.h5' % (step+1)
	g_model.save(CODE_PATH_DIR + filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

Train pix2pix models

In [0]:
def train(d_model, g_model, gan_model, dataset, n_epochs=6000, n_batch=1):
	# determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
	# unpack dataset
    trainA, trainB = dataset
	# calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
    print(n_steps)
    t1 = time()
    for i in range(30000):
		# select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
        if (i+1) % (10000) == 0:
            summarize_performance(i, g_model, dataset)
        if (i+1) % (3000) == 0:
            t2 = time() - t1
            print(t2)
            time_list.append(t2)
            with writer.as_default():
                  tf.summary.scalar("G_Loss", g_loss, step = i+1)
                  tf.summary.scalar("D_Loss_Real", d_loss1, step = i+1)
                  tf.summary.scalar("D_Loss_Fake", d_loss2, step = i+1)
                  writer.flush()

Load image data

In [17]:
dataset = load_real_samples(FILENAME_OPEN_PATH) 
print('Loaded', dataset[0].shape, dataset[1].shape)

Loaded (410, 256, 256, 3) (410, 256, 256, 3)


Define input shape based on the loaded dataset

In [0]:
image_shape = dataset[0].shape[1:]

Define the models

In [19]:
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

Discriminator image_shape: (256, 256, 3)
Generator image_shape: (256, 256, 3)


Define the composite model

In [0]:
gan_model = define_gan(g_model, d_model, image_shape)

In [0]:
writer = tf.summary.create_file_writer(LOGDIR)

Train model

In [22]:
time_list = []
train(d_model, g_model, gan_model, dataset)
for i in range(len(time_list)):
    print(str(i + 1) + " epoch took: " + str(time_list[i]))

2460000


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
>25015, d1[0.032] d2[0.037] g[10.897]
>25016, d1[0.000] d2[0.011] g[10.635]
>25017, d1[0.017] d2[0.007] g[17.834]
>25018, d1[0.006] d2[0.002] g[18.357]
>25019, d1[0.027] d2[0.006] g[13.895]
>25020, d1[0.000] d2[0.009] g[18.407]
>25021, d1[0.000] d2[0.051] g[13.331]
>25022, d1[0.000] d2[0.028] g[58.701]
>25023, d1[0.000] d2[0.007] g[22.035]
>25024, d1[0.001] d2[0.421] g[55.356]
>25025, d1[0.451] d2[0.003] g[12.503]
>25026, d1[0.588] d2[0.037] g[11.737]
>25027, d1[0.000] d2[0.006] g[21.901]
>25028, d1[0.002] d2[0.154] g[13.869]
>25029, d1[0.000] d2[0.020] g[100.996]
>25030, d1[0.000] d2[0.010] g[10.970]
>25031, d1[0.020] d2[0.041] g[15.580]
>25032, d1[0.002] d2[0.026] g[18.232]
>25033, d1[0.006] d2[0.007] g[22.583]
>25034, d1[0.026] d2[0.002] g[13.004]
>25035, d1[0.000] d2[0.020] g[25.420]
>25036, d1[0.000] d2[0.009] g[10.556]
>25037, d1[0.005] d2[0.066] g[10.603]
>25038, d1[0.026] d2[0.000] g[20.318]
>25039, d1[0.000] d2[0.017] g[18.826

Umounting Google Drive

In [0]:
drive.flush_and_unmount()